In [15]:
import os
import sqlite3
from pathlib import Path
import pandas as pd
import shutil

#gather data from a specific csv file and return as a pandas df
BASE = Path.cwd()
DATA = BASE / "training_data"
#Make sure training_data folder exists
DATA.mkdir(parents= True, exist_ok= True)

In [16]:
#Get the user path 
user_path = os.path.expanduser("~")
#Make the path to the kaggle json
downloads_path = os.path.join(user_path, 'Downloads/kaggle.json')

#Make the .kaggle folder in user
kaggle_folder = os.path.join(user_path, ".kaggle")
os.makedirs(kaggle_folder, exist_ok= True)

print(os.path.exists(downloads_path))

if os.path.exists(downloads_path):
    try:
        print("moving kaggle.json to user")
        #Make .kaggle folder path and check it exists
        
        #Move json to the folder
        shutil.move(downloads_path, kaggle_folder)
    except Exception as e:
        print(f"Error when trying to move kaggle.json: {e}")
else:
    #attempt to check if kaggle.json already exists
    final = os.path.join(kaggle_folder, "kaggle.json")
    if os.path.exists(final):
        print("kaggle json already exists in user")
    else:
        print("please download kaggle.json")

#print("Successfully moved json file")

False
kaggle json already exists in user


In [17]:
#TO USE THIS API you must have a .kaggle folder in your 'C:\NAME' directory -> then paste the kaggle.json authenticator
import kaggle
from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()

print(api.dataset_list_files('wordsforthewise/lending-club').files)

api.dataset_download_files('wordsforthewise/lending-club', path= DATA, unzip=True)

[{"ref": "", "datasetRef": null, "ownerRef": null, "name": "accepted_2007_to_2018Q4.csv.gz", "creationDate": "2019-12-17T23:30:10.124Z", "description": null, "fileType": null, "url": null, "totalBytes": 392582231, "columns": []}, {"ref": "", "datasetRef": null, "ownerRef": null, "name": "accepted_2007_to_2018q4.csv/accepted_2007_to_2018Q4.csv", "creationDate": "2019-04-10T18:09:09.272Z", "description": null, "fileType": null, "url": null, "totalBytes": 1675133810, "columns": []}, {"ref": "", "datasetRef": null, "ownerRef": null, "name": "rejected_2007_to_2018Q4.csv.gz", "creationDate": "2019-12-17T23:30:04.004Z", "description": null, "fileType": null, "url": null, "totalBytes": 255470782, "columns": []}, {"ref": "", "datasetRef": null, "ownerRef": null, "name": "rejected_2007_to_2018q4.csv/rejected_2007_to_2018Q4.csv", "creationDate": "2019-04-10T18:09:22.367Z", "description": null, "fileType": null, "url": null, "totalBytes": 1782281620, "columns": []}]
Dataset URL: https://www.kaggle

In [18]:
def retrieve_training_csv(): 
    """ Function that returns all the csv files in the training data folder as a dataframe object """ 
    csv_list = list(DATA.glob("**/*.csv")) 
    return_list = [] 
    print(csv_list)

    #traverse each item inthe data path, and if a file ending in .csv is found, turn it into a df and append to return list 
    for item in csv_list: 
        if os.path.isfile(item): 
            print(f"{item} is a file") 
            return_list.append(pd.read_csv(item)) 
        else: print(f"{item} is folder or dir")

    if return_list: 
        return return_list 
    else: 
        print(f"No csv files found in {DATA}") 
        return []

def get_dir_size(path): 
    """ Get directory size in MBs """ 
    total = 0 
    for dirpath, dirs, files in os.walk(path):
        for f in files:
            file_path = os.path.join(dirpath, f)
            
            try:
                total += os.path.getsize(file_path) 
                print(f"{total/1000000} MB")
            except: 
                continue 
        return total/1000000
    
def delete_large_files(): 
    """ Function that deletes the large files downloaded from kaggle to save space 
    Deletes files to prevent any storage errors when pushing code to github """

    for paths in DATA.glob("**/*"): 
        if os.path.isdir(paths): 
            try:
                size = get_dir_size(paths) 
                print(f"size of {paths} is {size}")

                if size > 100: 
                    print(f"deleting {paths}") 
                    shutil.rmtree(paths) 

            except Exception as e:
                print(f"Error when trying to get size or delete file: {e}")
        else:
            if paths.suffix == ".gz":
                os.remove(paths)
                #print(paths)
        

df_list = retrieve_training_csv()

[WindowsPath('c:/Users/Saul/Desktop/Credit Risk Assessment/credit-risk-assessment/ETL/training_data/accepted_2007_to_2018q4.csv'), WindowsPath('c:/Users/Saul/Desktop/Credit Risk Assessment/credit-risk-assessment/ETL/training_data/rejected_2007_to_2018q4.csv'), WindowsPath('c:/Users/Saul/Desktop/Credit Risk Assessment/credit-risk-assessment/ETL/training_data/accepted_2007_to_2018q4.csv/accepted_2007_to_2018Q4.csv'), WindowsPath('c:/Users/Saul/Desktop/Credit Risk Assessment/credit-risk-assessment/ETL/training_data/rejected_2007_to_2018q4.csv/rejected_2007_to_2018Q4.csv'), WindowsPath('c:/Users/Saul/Desktop/Credit Risk Assessment/credit-risk-assessment/ETL/training_data/workspace/loan_data.csv')]
c:\Users\Saul\Desktop\Credit Risk Assessment\credit-risk-assessment\ETL\training_data\accepted_2007_to_2018q4.csv is folder or dir
c:\Users\Saul\Desktop\Credit Risk Assessment\credit-risk-assessment\ETL\training_data\rejected_2007_to_2018q4.csv is folder or dir
c:\Users\Saul\Desktop\Credit Risk A

C:\Users\Saul\AppData\Local\Temp\ipykernel_22428\3710330960.py:11: DtypeWarning: Columns (0,19,49,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types. Specify dtype option on import or set low_memory=False.
  return_list.append(pd.read_csv(item))


c:\Users\Saul\Desktop\Credit Risk Assessment\credit-risk-assessment\ETL\training_data\rejected_2007_to_2018q4.csv\rejected_2007_to_2018Q4.csv is a file
c:\Users\Saul\Desktop\Credit Risk Assessment\credit-risk-assessment\ETL\training_data\workspace\loan_data.csv is a file


In [19]:
#create different pointers to the csvs in the list for easier access
raw_accepted_loans = df_list[0]
raw_rejected_loans = df_list[1]
raw_loans_paid_info = df_list[2]

In [20]:
raw_accepted_loans.head(10)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,68407277,NaN,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,68355089,NaN,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,68341763,NaN,20000.0,20000.0,20000.0,60 months,10.78,432.66,B,B4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
3,66310712,NaN,35000.0,35000.0,35000.0,60 months,14.85,829.90,C,C5,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
4,68476807,NaN,10400.0,10400.0,10400.0,60 months,22.45,289.91,F,F1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
5,68426831,NaN,11950.0,11950.0,11950.0,36 months,13.44,405.18,C,C3,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
6,68476668,NaN,20000.0,20000.0,20000.0,36 months,9.17,637.58,B,B2,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
7,67275481,NaN,20000.0,20000.0,20000.0,36 months,8.49,631.26,B,B1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
8,68466926,NaN,10000.0,10000.0,10000.0,36 months,6.49,306.45,A,A2,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
9,68616873,NaN,8000.0,8000.0,8000.0,36 months,11.48,263.74,B,B5,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
al_columns = list(raw_accepted_loans.columns)
print(al_columns)
raw_accepted_loans['loan_status'].head(10)
num_cols = [
    "id", "loan_amnt", "funded_amnt", "term", "int_rate", "installment", "annual_inc",
    "dti", "delinq_2yrs", "fico_range_low", "fico_range_high", "inq_last_6mths",
    "open_acc", "revol_bal", "revol_util", "total_acc", "pub_rec_bankruptcies"
]

text_cols = [
    "home_ownership", "loan_status", "purpose", "application_type", "verification_status"
]


"""
raw_specific_al = raw_accepted_loans[target_columns]
raw_specific_al.head(10).T
"""

['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title', 'emp_length', 'home_ownership', 'annual_inc', 'verification_status', 'issue_d', 'loan_status', 'pymnt_plan', 'url', 'desc', 'purpose', 'title', 'zip_code', 'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line', 'fico_range_low', 'fico_range_high', 'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'initial_list_status', 'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'last_pymnt_d', 'last_pymnt_amnt', 'next_pymnt_d', 'last_credit_pull_d', 'last_fico_range_high', 'last_fico_range_low', 'collections_12_mths_ex_med', 'mths_since_last_major_derog', 'policy_code', 'application_type', 'annual_inc_joint', 'dti_joint', 'verification_status_joint', 'acc_now_delinq',

'\nraw_specific_al = raw_accepted_loans[target_columns]\nraw_specific_al.head(10).T\n'

**Clean Accepted Loans Data**
- Provide a data summary
- Standardize all data in the columns to reasonable types (ex: object -> float)
- Clean up any bad data (missing values, duplicates, etc)


In [22]:
#only find columns with valid ids (not string or empty)
cleaned_accepted_loans = raw_accepted_loans[num_cols + text_cols]
cleaned_accepted_loans = cleaned_accepted_loans[pd.to_numeric(cleaned_accepted_loans['id'], errors='coerce').notna()]

#convert id column to int instead of obj
cleaned_accepted_loans['id'] = cleaned_accepted_loans['id'].astype('int64')

In [23]:
#fix the the term column to make it float (and in months)
cleaned_accepted_loans.rename(columns = {'term': 'term_months'}, inplace = True)
num_cols[3] = 'term_months'
cleaned_accepted_loans['term_months'] = cleaned_accepted_loans['term_months'].str.replace(" months", "")
cleaned_accepted_loans['term_months'] = cleaned_accepted_loans['term_months'].astype('int64')

In [24]:
cleaned_accepted_loans.isna().sum()

id                         0
loan_amnt                  0
funded_amnt                0
term_months                0
int_rate                   0
installment                0
annual_inc                 4
dti                     1711
delinq_2yrs               29
fico_range_low             0
fico_range_high            0
inq_last_6mths            30
open_acc                  29
revol_bal                  0
revol_util              1802
total_acc                 29
pub_rec_bankruptcies    1365
home_ownership             0
loan_status                0
purpose                    0
application_type           0
verification_status        0
dtype: int64

In [25]:
#find any na rows and fill them as needed
for rows in num_cols:
    cleaned_accepted_loans.loc[:, rows] = cleaned_accepted_loans[rows].fillna(cleaned_accepted_loans[rows].median())

cleaned_accepted_loans.isna().sum()


id                      0
loan_amnt               0
funded_amnt             0
term_months             0
int_rate                0
installment             0
annual_inc              0
dti                     0
delinq_2yrs             0
fico_range_low          0
fico_range_high         0
inq_last_6mths          0
open_acc                0
revol_bal               0
revol_util              0
total_acc               0
pub_rec_bankruptcies    0
home_ownership          0
loan_status             0
purpose                 0
application_type        0
verification_status     0
dtype: int64

In [26]:
#Get a sample of the cleaned up 
sampled_accepted_loans = cleaned_accepted_loans.sample(200000, random_state = 821)
sampled_accepted_loans

,id,loan_amnt,funded_amnt,term_months,int_rate,installment,annual_inc,dti,delinq_2yrs,fico_range_low,...,open_acc,revol_bal,revol_util,total_acc,pub_rec_bankruptcies,home_ownership,loan_status,purpose,application_type,verification_status
1488822,135703522,6500.0,6500.0,36,20.39,242.86,47000.0,10.29,0.0,660.0,...,11.0,5856.0,43.1,27.0,1.0,MORTGAGE,Current,house,Individual,Not Verified
567573,117605717,20000.0,20000.0,36,9.44,640.10,156000.0,26.65,0.0,740.0,...,20.0,20106.0,40.9,32.0,0.0,MORTGAGE,Current,home_improvement,Individual,Not Verified
1933073,1158084,26400.0,26400.0,60,9.76,557.81,90000.0,18.91,0.0,770.0,...,9.0,13856.0,20.6,24.0,0.0,MORTGAGE,Fully Paid,debt_consolidation,Individual,Verified
539548,119384164,8000.0,8000.0,36,16.02,281.34,38000.0,8.28,0.0,660.0,...,3.0,3399.0,75.5,12.0,0.0,OWN,Current,major_purchase,Individual,Source Verified
1057196,71806132,28000.0,28000.0,60,26.57,847.81,120000.0,24.77,0.0,680.0,...,12.0,65167.0,76.5,28.0,0.0,MORTGAGE,Charged Off,debt_consolidation,Individual,Source Verified
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
957236,105781229,18500.0,18500.0,36,6.99,571.15,81528.0,13.90,0.0,750.0,...,13.0,16760.0,36.0,26.0,0.0,MORTGAGE,Current,debt_consolidation,Individual,Source Verified
340434,43659370,20800.0,20800.0,60,10.99,452.14,91248.0,23.19,0.0,740.0,...,19.0,32518.0,33.7,46.0,0.0,MORTGAGE,Fully Paid,debt_consolidation,Individual,Source Verified
52235,63215769,12000.0,12000.0,36,7.89,375.43,70000.0,22.65,0.0,705.0,...,19.0,17132.0,58.7,39.0,0.0,RENT,Fully Paid,credit_card,Individual,Source Verified
2119521,121932789,25000.0,25000.0,60,11.99,555.99,81000.0,10.92,0.0,705.0,...,13.0,13651.0,41.9,22.0,0.0,RENT,Current,debt_consolidation,Joint App,Not Verified


**Clean Rejected Loan Data**

In [27]:
cleaned_rejected_loans = raw_rejected_loans.copy()
cleaned_rejected_loans.isna().sum()

Amount Requested               0
Application Date               0
Loan Title                  1305
Risk_Score              18497630
Debt-To-Income Ratio           0
Zip Code                     293
State                         22
Employment Length         951355
Policy Code                  918
dtype: int64

In [28]:
#Drop unneeded columns 
cleaned_rejected_loans = cleaned_rejected_loans.drop(columns=['Risk_Score', 'Zip Code', 'State', 'Policy Code', 'Employment Length'])
rl_cols = list(cleaned_rejected_loans.columns)
print(rl_cols)

cleaned_rejected_loans.dtypes

['Amount Requested', 'Application Date', 'Loan Title', 'Debt-To-Income Ratio']


Amount Requested        float64
Application Date         object
Loan Title               object
Debt-To-Income Ratio     object
dtype: object

In [29]:
print(type(cleaned_rejected_loans['Amount Requested'].dtype))

<class 'numpy.dtypes.Float64DType'>


In [30]:
#Fix any missing values in the rejection loans
for rows in rl_cols:
    #fix the rows that are of float type
    if cleaned_rejected_loans[rows].dtypes == 'float64':
        print(f"replacing {rows} with median float")
        cleaned_rejected_loans.loc[:, rows] = cleaned_rejected_loans[rows].fillna(cleaned_rejected_loans[rows].median())
    else:
        #get the highest repeated string and replace N/A's with that string
        print(f"replacing {rows} na's with most repeated string")
        replacement_string = cleaned_rejected_loans[rows].value_counts().reset_index().at[0,rows]
        cleaned_rejected_loans.loc[:, rows] = cleaned_rejected_loans[rows].fillna(replacement_string)

replacing Amount Requested with median float
replacing Application Date na's with most repeated string
replacing Loan Title na's with most repeated string
replacing Debt-To-Income Ratio na's with most repeated string


In [33]:
cleaned_rejected_loans.isna().sum()
print(len(cleaned_rejected_loans))

cleaned_rejected_loans.head(10)

27648741


,Amount Requested,Application Date,Loan Title,Debt-To-Income Ratio
0,1000.0,2007-05-26,Wedding Covered but No Honeymoon,10%
1,1000.0,2007-05-26,Consolidating Debt,10%
2,11000.0,2007-05-27,Want to consolidate my debt,10%
3,6000.0,2007-05-27,waksman,38.64%
4,1500.0,2007-05-27,mdrigo,9.43%
5,15000.0,2007-05-27,Trinfiniti,0%
6,10000.0,2007-05-27,NOTIFYi Inc,10%
7,3900.0,2007-05-27,For Justin.,10%
8,3000.0,2007-05-28,title?,10%
9,2500.0,2007-05-28,timgerst,11.76%


In [32]:
#delete_large_files()